In [1]:
import tensorflow as tf
import seaborn as sns
import numpy as np

from PIL import Image
import glob
from collections import defaultdict
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
IMG_SIZE = (113, 240)
def pixels_from_path(file_path):
    im = Image.open(file_path)
    im = im.resize(IMG_SIZE)
    np_im = np.array(im)
    return np_im

Total number of images = 8192 

Training set- 6554 images

Testing set- 1638

In [22]:
left_path = "D:/Nauka/Magisterskie/!sem3/MSI/projekt/lewo/"
right_path = "D:/Nauka/Magisterskie/!sem3/MSI/projekt/prawo/"

In [36]:
sample_size = 3277
left_train_set = np.asarray([pixels_from_path(left) for left in glob.glob(left_path + "*")[:sample_size]])

In [37]:
len(left_train_set)

3277

In [54]:
right_train_set = np.asarray([pixels_from_path(right) for right in glob.glob(right_path +"*")[:sample_size]])

In [55]:
len(right_train_set)

3277

In [40]:
test_size = 819
left_test_set = np.asarray([pixels_from_path(left) for left in glob.glob(left_path + "*")[-test_size:]])
right_test_set = np.asarray([pixels_from_path(right) for right in glob.glob(right_path + "*")[-test_size:]])

left 1

right 0

In [56]:
x_train = np.concatenate([left_train_set, right_train_set])
labels_train = np.asarray([1 for _ in range(sample_size)] + [0 for _ in range(sample_size)])

In [42]:
x_test = np.concatenate([left_test_set, right_test_set])
labels_test = np.asarray([1 for _ in range(test_size)] + [0 for _ in range (test_size)])

In [57]:
x_train.shape

(6554, 240, 113, 3)

In [58]:
labels_train.shape

(6554,)

In [59]:
x_test.shape

(1638, 240, 113, 3)

In [60]:
labels_test.shape

(1638,)

Initial convolutional model

In [62]:
fc_layer_size = 256
img_size = IMG_SIZE

conv_inputs = keras.Input(shape = (img_size[1], img_size[0], 3), name='robo_cam_frame')

conv_layer = layers.Conv2D(128, kernel_size=3, activation='relu')(conv_inputs)
conv_layer = layers.MaxPool2D(pool_size=(2,2))(conv_layer)

conv_x = layers.Flatten(name = 'flattened_features')(conv_layer)

conv_x = layers.Dense(fc_layer_size, activation='relu', name='first_layer')(conv_x)
conv_x = layers.Dense(fc_layer_size, activation='relu', name='second_layer')(conv_x)
conv_outputs =  layers.Dense(1, activation='sigmoid', name='class')(conv_x)

conv_model = keras.Model(inputs = conv_inputs, outputs = conv_outputs)

In [68]:
customAdam = keras.optimizers.Adam(lr = 1e-6)
conv_model.compile(optimizer=customAdam, loss="binary_crossentropy", metrics = ["binary_crossentropy", "mean_squared_error"])

In [65]:
history = conv_model.fit(x_train, labels_train, batch_size=32, shuffle=True, epochs=5, validation_data=(x_test, labels_test))

Train on 6554 samples, validate on 1638 samples
Epoch 1/5
6554/6554 [==============================] - 498s 76ms/sample - loss: 0.1854 - binary_crossentropy: 0.1854 - mean_squared_error: 0.0490 - val_loss: 0.0951 - val_binary_crossentropy: 0.0951 - val_mean_squared_error: 0.0262
Epoch 2/5
6554/6554 [==============================] - 471s 72ms/sample - loss: 0.1247 - binary_crossentropy: 0.1247 - mean_squared_error: 0.0327 - val_loss: 0.0913 - val_binary_crossentropy: 0.0913 - val_mean_squared_error: 0.0229
Epoch 3/5
6554/6554 [==============================] - 471s 72ms/sample - loss: 0.1338 - binary_crossentropy: 0.1338 - mean_squared_error: 0.0317 - val_loss: 0.0882 - val_binary_crossentropy: 0.0882 - val_mean_squared_error: 0.0245
Epoch 4/5
6554/6554 [==============================] - 460s 70ms/sample - loss: 0.0992 - binary_crossentropy: 0.0992 - mean_squared_error: 0.0238 - val_loss: 0.1285 - val_binary_crossentropy: 0.1285 - val_mean_squared_error: 0.0306
Epoch 5/5
6554/6554 [===

In [66]:
preds = conv_model.predict(x_test)
preds = np.asarray([pred[0] for pred in preds])
np.corrcoef(preds, labels_test)

array([[1.        , 0.91396279],
       [0.91396279, 1.        ]])

Bigger convolutional part

In [69]:
fc_layer_size = 256
img_size = IMG_SIZE

big_conv_inputs = keras.Input(shape = (img_size[1], img_size[0], 3), name='robo_cam_frame')

big_conv_layer = layers.Conv2D(64, kernel_size=3, activation='relu')(big_conv_inputs)
big_conv_layer = layers.MaxPool2D(pool_size=(2,2))(big_conv_layer)

big_conv_layer = layers.Conv2D(128, kernel_size=3, activation='relu')(big_conv_inputs)
big_conv_layer = layers.MaxPool2D(pool_size=(2,2))(big_conv_layer)

big_conv_x = layers.Flatten(name = 'flattened_features')(big_conv_layer)

big_conv_x = layers.Dense(fc_layer_size, activation='relu', name='first_layer')(big_conv_x)
big_conv_x = layers.Dense(fc_layer_size, activation='relu', name='second_layer')(big_conv_x)
big_conv_outputs =  layers.Dense(1, activation='sigmoid', name='class')(big_conv_x)

big_conv_model = keras.Model(inputs = big_conv_inputs, outputs = big_conv_outputs)

In [70]:
customAdam = keras.optimizers.Adam(lr = 1e-6)
big_conv_model.compile(optimizer=customAdam, loss="binary_crossentropy", metrics = ["binary_crossentropy", "mean_squared_error"])

In [71]:
history = big_conv_model.fit(x_train, labels_train, batch_size=32, shuffle=True, epochs=5, validation_data=(x_test, labels_test))

Train on 6554 samples, validate on 1638 samples
Epoch 1/5
6554/6554 [==============================] - 505s 77ms/sample - loss: 0.4289 - binary_crossentropy: 0.4289 - mean_squared_error: 0.0731 - val_loss: 0.4543 - val_binary_crossentropy: 0.4543 - val_mean_squared_error: 0.1037
Epoch 2/5
6554/6554 [==============================] - 470s 72ms/sample - loss: 0.1337 - binary_crossentropy: 0.1337 - mean_squared_error: 0.0326 - val_loss: 0.3246 - val_binary_crossentropy: 0.3246 - val_mean_squared_error: 0.0751
Epoch 3/5
6554/6554 [==============================] - 448s 68ms/sample - loss: 0.2140 - binary_crossentropy: 0.2140 - mean_squared_error: 0.0431 - val_loss: 0.1919 - val_binary_crossentropy: 0.1919 - val_mean_squared_error: 0.0391
Epoch 4/5
6554/6554 [==============================] - 452s 69ms/sample - loss: 0.1321 - binary_crossentropy: 0.1321 - mean_squared_error: 0.0290 - val_loss: 0.0598 - val_binary_crossentropy: 0.0598 - val_mean_squared_error: 0.0159
Epoch 5/5
6554/6554 [===

In [73]:
preds_2 = big_conv_model.predict(x_test)
preds_2 = np.asarray([pred[0] for pred in preds_2])
np.corrcoef(preds_2, labels_test)

array([[1.        , 0.95585357],
       [0.95585357, 1.        ]])

Bigger NN part

In [82]:
fc_layer_size = 512
img_size = IMG_SIZE

bigger_conv_inputs = keras.Input(shape=(img_size[1], img_size[0], 3), name='robo_cam_frame')

bigger_conv_layer = layers.Conv2D(64, kernel_size = 3, activation='relu')(bigger_conv_inputs)
bigger_conv_layer = layers.MaxPool2D(pool_size=(2,2))(bigger_conv_layer)

bigger_conv_layer = layers.Conv2D(128, kernel_size = 3, activation='relu')(bigger_conv_layer)
bigger_conv_layer = layers.MaxPool2D(pool_size=(2,2))(bigger_conv_layer)

bigger_conv_x = layers.Flatten(name = 'flattened_features')(bigger_conv_layer)

bigger_conv_x = layers.Dense(fc_layer_size, activation='relu', name='first_layer')(bigger_conv_x)
bigger_conv_x = layers.Dense(fc_layer_size, activation='relu', name='second_layer')(bigger_conv_x)
bigger_conv_outputs = layers.Dense(1, activation='sigmoid', name='class')(bigger_conv_x)

bigger_conv_model = keras.Model(inputs = bigger_conv_inputs, outputs = bigger_conv_outputs)

In [84]:
customAdam = keras.optimizers.Adam(lr = 1e-6)
bigger_conv_model.compile(optimizer=customAdam, loss="binary_crossentropy", metrics = ["binary_crossentropy", "mean_squared_error"])

In [85]:
history = bigger_conv_model.fit(x_train, labels_train, batch_size=32, shuffle=True, epochs=5, validation_data=(x_test, labels_test))

Train on 6554 samples, validate on 1638 samples
Epoch 1/5
6554/6554 [==============================] - 347s 53ms/sample - loss: 0.5950 - binary_crossentropy: 0.5950 - mean_squared_error: 0.0935 - val_loss: 0.0991 - val_binary_crossentropy: 0.0991 - val_mean_squared_error: 0.0287
Epoch 2/5
6554/6554 [==============================] - 341s 52ms/sample - loss: 0.1074 - binary_crossentropy: 0.1074 - mean_squared_error: 0.0306 - val_loss: 0.0572 - val_binary_crossentropy: 0.0572 - val_mean_squared_error: 0.0166
Epoch 3/5
6554/6554 [==============================] - 327s 50ms/sample - loss: 0.0750 - binary_crossentropy: 0.0750 - mean_squared_error: 0.0210 - val_loss: 0.0539 - val_binary_crossentropy: 0.0539 - val_mean_squared_error: 0.0158
Epoch 4/5
6554/6554 [==============================] - 325s 50ms/sample - loss: 0.0593 - binary_crossentropy: 0.0593 - mean_squared_error: 0.0158 - val_loss: 0.0508 - val_binary_crossentropy: 0.0508 - val_mean_squared_error: 0.0148
Epoch 5/5
6554/6554 [===

In [87]:
preds_3 = bigger_conv_model.predict(x_test)
preds_3 = np.asarray([pred[0] for pred in preds_3])
np.corrcoef(preds_3, labels_test)

array([[1.        , 0.97810496],
       [0.97810496, 1.        ]])